In [1]:
import numpy as np
import pandas as pd
from collections import Counter
import statistics

### import GEO reference data

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# import GEO reference data
geoSamples = pd.read_csv(r'/content/drive/MyDrive/reusability/samples.csv', low_memory = False)
geoSeries = pd.read_csv(r'/content/drive/MyDrive/reusability/series.csv', low_memory = False)
geoSeries.rename(columns = {'Accession':'Series'}, inplace = True)

geoReference = pd.merge(geoSamples, geoSeries, how = 'outer', on = 'Series')[['Series', 'Accession', 'Platform']].drop_duplicates()

In [4]:
geoReference

,Series,Accession,Platform
0,GSE506,GSM1,GPL4
1,GSE506,GSM2,GPL4
2,GSE462,GSM3,GPL5
3,GSE462,GSM4,GPL5
4,GSE462,GSM5,GPL5
...,...,...,...
1243682,GSE267405,NaN,NaN
1243683,GSE267438,NaN,NaN
1243684,GSE267595,NaN,NaN
1243685,GSE267653,NaN,NaN


### import SRA reference data

In [5]:
# import SRA reference data
sraReference_all = pd.read_csv(r'/content/drive/MyDrive/reusability/sra_complete_runs_unique.csv', low_memory=False, quoting=3, on_bad_lines='skip')

In [6]:
sraReference=sraReference_all
sraReference.head()

,Run,ReleaseDate,bases,avgLength,Experiment,LibraryStrategy,LibrarySelection,LibrarySource,Platform,Model,SRAStudy,BioProject,Sample,ScientificName,Submission
0,SRR7968967,2018-10-08,4538740500,150,SRX4802362,ChIP-Seq,ChIP,GENOMIC,ILLUMINA,Illumina HiSeq 4000,SRP163642,PRJNA494833,SRS3881321,Homo sapiens,SRA789647
1,SRR22439147,2023-12-01,16725226500,300,SRX18408150,RNA-Seq,RANDOM PCR,TRANSCRIPTOMIC,ILLUMINA,Illumina NovaSeq 6000,SRP410343,PRJNA899343,SRS15891322,Homo sapiens,SRA1547816
2,SRR22459658,2023-12-01,7968134424,152,SRX18426201,miRNA-Seq,RT-PCR,TRANSCRIPTOMIC,ILLUMINA,Illumina NovaSeq 6000,SRP410343,PRJNA899343,SRS15891322,Homo sapiens,SRA1549603
3,SRR22439146,2023-12-01,15242483700,300,SRX18408151,RNA-Seq,RANDOM PCR,TRANSCRIPTOMIC,ILLUMINA,Illumina NovaSeq 6000,SRP410343,PRJNA899343,SRS15891323,Homo sapiens,SRA1547816
4,SRR22459657,2023-12-01,5266201272,152,SRX18426202,miRNA-Seq,RT-PCR,TRANSCRIPTOMIC,ILLUMINA,Illumina NovaSeq 6000,SRP410343,PRJNA899343,SRS15891323,Homo sapiens,SRA1549603


In [ ]:
#sraReference.columns = ['SRAStudy', 'Run', 'Experiment', 'BioProject', 'Submission', 'Sample', 'ReleaseDate','Model', 'LibraryStrategy', 'ScientificName', 'bases', 'avgLength']

In [7]:
# for collecting desired factors step
sraAttributes = sraReference

In [8]:
sraReference = sraReference[['SRAStudy', 'Run', 'Experiment', 'BioProject', 'Submission', 'Sample']]

In [ ]:
sraReference.head()

,SRAStudy,Run,Experiment,BioProject,Submission,Sample
0,SRP163642,SRR7968967,SRX4802362,PRJNA494833,SRA789647,SRS3881321
1,SRP410343,SRR22439147,SRX18408150,PRJNA899343,SRA1547816,SRS15891322
2,SRP410343,SRR22459658,SRX18426201,PRJNA899343,SRA1549603,SRS15891322
3,SRP410343,SRR22439146,SRX18408151,PRJNA899343,SRA1547816,SRS15891323
4,SRP410343,SRR22459657,SRX18426202,PRJNA899343,SRA1549603,SRS15891323


In [9]:
# import data scraped from PubMed
pmcData = pd.read_csv(r'/content/drive/MyDrive/reusability/preFilterMatrix_combined.csv')

In [10]:
pmcData2 = pmcData
pmcData.head()

,journal,pmc_ID,accession,pmc_date
0,J_Invest_Dermatol,PMC3305850,GPL1,2012-04-26
1,Genetics,PMC3606103,GSE44020,2013-04-01
2,Genetics,PMC3606103,SRX217718,2013-04-01
3,Genetics,PMC3606103,GSM107672,2013-04-01
4,Nature,PMC3936678,GSE45494,2013-07-11


### Merge GEO accessions with reference data, convert to Series

In [11]:
# match series
Series_Merge = pd.merge(pmcData, geoReference['Series'].drop_duplicates(), how = 'left', left_on = 'accession', right_on = 'Series')
pmcData = Series_Merge.rename(columns = {'Series': 'Series_result'})
pmcData.head()

,journal,pmc_ID,accession,pmc_date,Series_result
0,J_Invest_Dermatol,PMC3305850,GPL1,2012-04-26,NaN
1,Genetics,PMC3606103,GSE44020,2013-04-01,GSE44020
2,Genetics,PMC3606103,SRX217718,2013-04-01,NaN
3,Genetics,PMC3606103,GSM107672,2013-04-01,NaN
4,Nature,PMC3936678,GSE45494,2013-07-11,GSE45494


In [12]:
# match each other GEO ID (sample, platform, and dataset)
for subject in ['Accession', 'Platform']:
    pmcData = pd.merge(pmcData, geoReference[['Series', subject]].drop_duplicates(subset = subject), how = 'left',
            left_on = 'accession', right_on = subject)
    label = subject + '_result'
    pmcData = pmcData.rename(columns = {'Series': label})

pmcData.head()

,journal,pmc_ID,accession,pmc_date,Series_result,Accession_result,Accession,Platform_result,Platform
0,J_Invest_Dermatol,PMC3305850,GPL1,2012-04-26,NaN,NaN,NaN,NaN,NaN
1,Genetics,PMC3606103,GSE44020,2013-04-01,GSE44020,NaN,NaN,NaN,NaN
2,Genetics,PMC3606103,SRX217718,2013-04-01,NaN,NaN,NaN,NaN,NaN
3,Genetics,PMC3606103,GSM107672,2013-04-01,NaN,GSE4765,GSM107672,NaN,NaN
4,Nature,PMC3936678,GSE45494,2013-07-11,GSE45494,NaN,NaN,NaN,NaN


In [13]:
# combine all GEO series match columns into one aggregate GEO series column, clean up
pmcData['geoSeries'] = pmcData['Series_result'].fillna(pmcData['Accession_result']).fillna(pmcData['Platform_result'])
pmcData = pmcData.drop(labels = ['Accession', 'Platform',
                                 'Series_result', 'Accession_result',
                                 'Platform_result'], axis = 1)
pmcData_geoMerged = pmcData

In [ ]:
pmcData_geoMerged.head()

,journal,pmc_ID,accession,pmc_date,geoSeries
0,J_Invest_Dermatol,PMC3305850,GPL1,2012-04-26,NaN
1,Genetics,PMC3606103,GSE44020,2013-04-01,GSE44020
2,Genetics,PMC3606103,SRX217718,2013-04-01,NaN
3,Genetics,PMC3606103,GSM107672,2013-04-01,GSE4765
4,Nature,PMC3936678,GSE45494,2013-07-11,GSE45494


### merge SRA accessions with reference data, convert to Study

In [14]:
# match SRA Study IDs
Study_Merge = pd.merge(pmcData_geoMerged, sraReference['SRAStudy'].drop_duplicates(), how = 'left', left_on = 'accession', right_on = 'SRAStudy')
pmcData = Study_Merge.rename(columns = {'SRAStudy': 'Study_result'})

In [ ]:
pmcData.head()

,journal,pmc_ID,accession,pmc_date,geoSeries,Study_result
0,J_Invest_Dermatol,PMC3305850,GPL1,2012-04-26,NaN,NaN
1,Genetics,PMC3606103,GSE44020,2013-04-01,GSE44020,NaN
2,Genetics,PMC3606103,SRX217718,2013-04-01,NaN,NaN
3,Genetics,PMC3606103,GSM107672,2013-04-01,GSE4765,NaN
4,Nature,PMC3936678,GSE45494,2013-07-11,GSE45494,NaN


In [15]:
# match every other SRA IDs (Run, Experiment, BioProject, Submission, Sample)
for subject in ['Run', 'Experiment', 'BioProject', 'Submission', 'Sample']:
    pmcData = pd.merge(pmcData, sraReference[['SRAStudy', subject]].drop_duplicates(subset = [subject]), how = 'left', left_on = 'accession', right_on = subject)
    label = subject + '_result'
    pmcData = pmcData.rename(columns = {'SRAStudy': label})

In [16]:
# combine all SRA Study matches into one aggregate column, clean up
pmcData['sraStudy'] = pmcData['Study_result'].fillna(pmcData['Run_result']).fillna(pmcData['Experiment_result']).fillna(pmcData['BioProject_result']).fillna(pmcData['Submission_result']).fillna(pmcData['Sample_result'])
pmcData = pmcData.drop(labels = ['Run', 'Experiment', 'BioProject', 'Submission', 'Sample',
                                'Study_result', 'Run_result', 'Experiment_result', 'BioProject_result',
                                'Submission_result', 'Sample_result'], axis = 1)
pmcData_sraMerged = pmcData

In [17]:
pmcData_sraMerged.head()

,journal,pmc_ID,accession,pmc_date,geoSeries,sraStudy
0,J_Invest_Dermatol,PMC3305850,GPL1,2012-04-26,NaN,NaN
1,Genetics,PMC3606103,GSE44020,2013-04-01,GSE44020,NaN
2,Genetics,PMC3606103,SRX217718,2013-04-01,NaN,SRP017961
3,Genetics,PMC3606103,GSM107672,2013-04-01,GSE4765,NaN
4,Nature,PMC3936678,GSE45494,2013-07-11,GSE45494,NaN


In [18]:
# combine GEO Series hits and SRA study hits into one converted accession column
pmcData['converted_accession'] = pmcData_sraMerged['geoSeries'].fillna(pmcData_sraMerged['sraStudy'])
pmcData = pmcData.drop(labels = ['geoSeries', 'sraStudy'], axis = 1)
pmcData.head()

,journal,pmc_ID,accession,pmc_date,converted_accession
0,J_Invest_Dermatol,PMC3305850,GPL1,2012-04-26,NaN
1,Genetics,PMC3606103,GSE44020,2013-04-01,GSE44020
2,Genetics,PMC3606103,SRX217718,2013-04-01,SRP017961
3,Genetics,PMC3606103,GSM107672,2013-04-01,GSE4765
4,Nature,PMC3936678,GSE45494,2013-07-11,GSE45494


### data cleaning

In [19]:
# clean out garbage converted_accession entries, and rows that didn't map to a converted_accession
w = []
for a in pmcData.converted_accession:
    if type(a)==str:
        if a[0:3] != 'GSE' or a[1:3] != 'RP':
            w.append(a)
pmcData = pmcData[pmcData.converted_accession.isin(w)]
pmcData.head()

,journal,pmc_ID,accession,pmc_date,converted_accession
1,Genetics,PMC3606103,GSE44020,2013-04-01,GSE44020
2,Genetics,PMC3606103,SRX217718,2013-04-01,SRP017961
3,Genetics,PMC3606103,GSM107672,2013-04-01,GSE4765
4,Nature,PMC3936678,GSE45494,2013-07-11,GSE45494
5,Nature,PMC3936678,GSM110564,2013-07-11,GSE4918


In [ ]:
# perform QC with gold standard from Penn group (Casey + Kurt)
gsPMC = pd.read_table(r'/content/drive/MyDrive/Colab Notebooks/geo/pubmed_mappings.tsv')
gsPMC.columns = ["SRA_accession_code", "GEO_accession_code", "pm_ID", "pmc_ID"]
gsPMC.head()

,SRA_accession_code,GEO_accession_code,pm_ID,pmc_ID
0,SRP111833,GSE101341,29535194.0,PMC5850328
1,ERP108370,NaN,NaN,NaN
2,SRP062170,GSE71840,27780967.0,NaN
3,ERP114122,NaN,NaN,NaN
4,SRP162020,GSE120109,30692590.0,PMC6349857


In [ ]:
# QC step: de-duplicate datasets present in both SRA and GEO
gsPMC_acc = gsPMC[['SRA_accession_code', 'GEO_accession_code']].dropna().drop_duplicates()
gsPMC_acc.head()

,SRA_accession_code,GEO_accession_code
0,SRP111833,GSE101341
2,SRP062170,GSE71840
4,SRP162020,GSE120109
5,SRP041755,GSE57401
6,SRP153370,GSE117074


In [ ]:
ovAcc = pd.merge(pmcData, gsPMC_acc, how = 'left', left_on = 'converted_accession', right_on = 'SRA_accession_code')
ovAcc.head()

,journal,pmc_ID,accession,pmc_date,converted_accession,SRA_accession_code,GEO_accession_code
0,Genetics,PMC3606103,GSE44020,2013-04-01,GSE44020,NaN,NaN
1,Genetics,PMC3606103,SRX217718,2013-04-01,SRP017961,NaN,NaN
2,Genetics,PMC3606103,GSM107672,2013-04-01,GSE4765,NaN,NaN
3,Nature,PMC3936678,GSE45494,2013-07-11,GSE45494,NaN,NaN
4,Nature,PMC3936678,GSM110564,2013-07-11,GSE4918,NaN,NaN


In [ ]:
# count the number of SRA datasets also present in GEO
numDupSRA = len(ovAcc['SRA_accession_code']) - ovAcc['SRA_accession_code'].isna().sum()
print('duplicated SRA datasets: ' + str(numDupSRA))

duplicated SRA datasets: 6481


In [ ]:
# convert SRA ID of duplicated datasets to GEO ID
ovAccNA = ovAcc.loc[ovAcc['SRA_accession_code'].isna(), :]
ovAcc = ovAcc.loc[~ovAcc['SRA_accession_code'].isna(), :]
ovAcc['converted_accession'] = ovAcc['GEO_accession_code']
ovAccNA

<ipython-input-27-413f3090aab4>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ovAcc['converted_accession'] = ovAcc['GEO_accession_code']


,journal,pmc_ID,accession,pmc_date,converted_accession,SRA_accession_code,GEO_accession_code
0,Genetics,PMC3606103,GSE44020,2013-04-01,GSE44020,NaN,NaN
1,Genetics,PMC3606103,SRX217718,2013-04-01,SRP017961,NaN,NaN
2,Genetics,PMC3606103,GSM107672,2013-04-01,GSE4765,NaN,NaN
3,Nature,PMC3936678,GSE45494,2013-07-11,GSE45494,NaN,NaN
4,Nature,PMC3936678,GSM110564,2013-07-11,GSE4918,NaN,NaN
...,...,...,...,...,...,...,...
406978,BMC_Genomics,PMC1868757,GSE7364,2007-04-30,GSE7364,NaN,NaN
406979,BMC_Cancer,PMC1533844,GSE4127,2006-06-30,GSE4127,NaN,NaN
406980,PLoS_Genet,PMC1564424,GSE5405,2006-09-15,GSE5405,NaN,NaN
406981,Genome_Biol,PMC1929143,GSE1159,2007-05-08,GSE1159,NaN,NaN


In [ ]:
pmcData = pd.concat([ovAcc, ovAccNA], axis = 0)
pmcData = pmcData[['pmc_ID', 'accession', 'converted_accession', 'pmc_date']]
pmcData.head()

,pmc_ID,accession,converted_accession,pmc_date
658,PMC3879970,SRP008153,GSE32074,2013-11-18
659,PMC3879970,SRP003874,GSE24872,2013-11-18
660,PMC3879970,SRP009679,GSE34319,2013-11-18
1049,PMC3893718,GSE24843,E-GEOD-24843,2013-08-08
1122,PMC3921077,SRR207100,GSE29278,2014-01-20


In [20]:
# count GEO number and SRA number
GEO_num = 0
SRA_num = 0
for i in pmcData.converted_accession:
    if i[0:3] == 'GSE':
        GEO_num = GEO_num + 1
    else:
        SRA_num = SRA_num + 1
print('GEO number:', GEO_num)
print('SRA number:', SRA_num)

GEO number: 250705
SRA number: 156278


### Collect desired factors

In [21]:
sraAttributes.head()

,Run,ReleaseDate,bases,avgLength,Experiment,LibraryStrategy,LibrarySelection,LibrarySource,Platform,Model,SRAStudy,BioProject,Sample,ScientificName,Submission
0,SRR7968967,2018-10-08,4538740500,150,SRX4802362,ChIP-Seq,ChIP,GENOMIC,ILLUMINA,Illumina HiSeq 4000,SRP163642,PRJNA494833,SRS3881321,Homo sapiens,SRA789647
1,SRR22439147,2023-12-01,16725226500,300,SRX18408150,RNA-Seq,RANDOM PCR,TRANSCRIPTOMIC,ILLUMINA,Illumina NovaSeq 6000,SRP410343,PRJNA899343,SRS15891322,Homo sapiens,SRA1547816
2,SRR22459658,2023-12-01,7968134424,152,SRX18426201,miRNA-Seq,RT-PCR,TRANSCRIPTOMIC,ILLUMINA,Illumina NovaSeq 6000,SRP410343,PRJNA899343,SRS15891322,Homo sapiens,SRA1549603
3,SRR22439146,2023-12-01,15242483700,300,SRX18408151,RNA-Seq,RANDOM PCR,TRANSCRIPTOMIC,ILLUMINA,Illumina NovaSeq 6000,SRP410343,PRJNA899343,SRS15891323,Homo sapiens,SRA1547816
4,SRR22459657,2023-12-01,5266201272,152,SRX18426202,miRNA-Seq,RT-PCR,TRANSCRIPTOMIC,ILLUMINA,Illumina NovaSeq 6000,SRP410343,PRJNA899343,SRS15891323,Homo sapiens,SRA1549603


In [22]:
# Convert SRA dates to a universal format
sraAttributes['ReleaseDate'] = sraAttributes['ReleaseDate'].str[0:10]

In [23]:
sraAttributes.head()

,Run,ReleaseDate,bases,avgLength,Experiment,LibraryStrategy,LibrarySelection,LibrarySource,Platform,Model,SRAStudy,BioProject,Sample,ScientificName,Submission
0,SRR7968967,2018-10-08,4538740500,150,SRX4802362,ChIP-Seq,ChIP,GENOMIC,ILLUMINA,Illumina HiSeq 4000,SRP163642,PRJNA494833,SRS3881321,Homo sapiens,SRA789647
1,SRR22439147,2023-12-01,16725226500,300,SRX18408150,RNA-Seq,RANDOM PCR,TRANSCRIPTOMIC,ILLUMINA,Illumina NovaSeq 6000,SRP410343,PRJNA899343,SRS15891322,Homo sapiens,SRA1547816
2,SRR22459658,2023-12-01,7968134424,152,SRX18426201,miRNA-Seq,RT-PCR,TRANSCRIPTOMIC,ILLUMINA,Illumina NovaSeq 6000,SRP410343,PRJNA899343,SRS15891322,Homo sapiens,SRA1549603
3,SRR22439146,2023-12-01,15242483700,300,SRX18408151,RNA-Seq,RANDOM PCR,TRANSCRIPTOMIC,ILLUMINA,Illumina NovaSeq 6000,SRP410343,PRJNA899343,SRS15891323,Homo sapiens,SRA1547816
4,SRR22459657,2023-12-01,5266201272,152,SRX18426202,miRNA-Seq,RT-PCR,TRANSCRIPTOMIC,ILLUMINA,Illumina NovaSeq 6000,SRP410343,PRJNA899343,SRS15891323,Homo sapiens,SRA1549603


In [24]:
# Define functions to convert GEO dates to a universal format
def strToMonth(m):
    if(m == 'Jan'):
        return '01'
    elif(m == 'Feb'):
        return '02'
    elif(m == 'Mar'):
        return '03'
    elif(m == 'Apr'):
        return '04'
    elif(m == 'May'):
        return '05'
    elif(m == 'Jun'):
        return '06'
    elif(m == 'Jul'):
        return '07'
    elif(m == 'Aug'):
        return '08'
    elif(m == 'Sep'):
        return '09'
    elif(m == 'Oct'):
        return '10'
    elif(m == 'Nov'):
        return '11'
    elif(m == 'Dec'):
        return '12'
    else:
        return(np.NaN)

def convGEODate(d):
    if(type(d) == str):
        mon = strToMonth(d[0:3])
        day = d[4:6]
        yr = d[8:12]
        return yr + '-' + mon + '-' + day
    else:
        return np.NaN

In [25]:
# import GEO attribute data and add Series column
geoPlatforms = pd.read_csv('/content/drive/MyDrive/reusability/platforms.csv')
geoPlatforms.rename(columns={'Accession':'Platform'}, inplace = True)
techByPlatform = geoPlatforms[['Platform', 'Technology']]

# allData contains metadata matched to GEO series, but lacks 'Technology' column
allData = pd.merge(geoSamples, geoSeries, how = 'outer')
geoAttributes = pd.merge(allData, techByPlatform, how = 'left', on = 'Platform')

In [26]:
geoAttributes.head()

,Accession,Title,Sample Type,Taxonomy,Channels,Platform,Series,Supplementary Types,Supplementary Links,SRA Accession,Contact,Release Date,Series Type,Sample Count,Datasets,PubMed ID,Technology
0,GSM1,Foreskin Fibroblasts,SAGE,Homo sapiens,1.0,GPL4,GSE506,NaN,NaN,NaN,Marc Kenzelmann,2000-09-28,NaN,NaN,NaN,NaN,SAGE NlaIII
1,GSM2,HCMV-infected foreskin fibroblasts,SAGE,Homo sapiens,1.0,GPL4,GSE506,NaN,NaN,NaN,Marc Kenzelmann,2000-09-28,NaN,NaN,NaN,NaN,SAGE NlaIII
2,GSM3,testis a,RNA,Drosophila melanogaster,1.0,GPL5,GSE462,NaN,NaN,NaN,Brian Oliver,2000-10-18,NaN,NaN,NaN,NaN,spotted DNA/cDNA
3,GSM4,testis b,RNA,Drosophila melanogaster,1.0,GPL5,GSE462,NaN,NaN,NaN,Brian Oliver,2000-10-18,NaN,NaN,NaN,NaN,spotted DNA/cDNA
4,GSM5,male a,RNA,Drosophila melanogaster,1.0,GPL5,GSE462,NaN,NaN,NaN,Brian Oliver,2000-10-18,NaN,NaN,NaN,NaN,spotted DNA/cDNA


In [ ]:
# convert GEO dates to universal format
#dates = []
#for i in geoAttributes['Release Date']:
#    dates.append(convGEODate(i))
#geoAttributes['Release_Date'] = dates

In [27]:
geoAttributes.rename(columns = {'Release Date':'Release_Date'}, inplace = True)

In [28]:
geoAttributes.head()

,Accession,Title,Sample Type,Taxonomy,Channels,Platform,Series,Supplementary Types,Supplementary Links,SRA Accession,Contact,Release_Date,Series Type,Sample Count,Datasets,PubMed ID,Technology
0,GSM1,Foreskin Fibroblasts,SAGE,Homo sapiens,1.0,GPL4,GSE506,NaN,NaN,NaN,Marc Kenzelmann,2000-09-28,NaN,NaN,NaN,NaN,SAGE NlaIII
1,GSM2,HCMV-infected foreskin fibroblasts,SAGE,Homo sapiens,1.0,GPL4,GSE506,NaN,NaN,NaN,Marc Kenzelmann,2000-09-28,NaN,NaN,NaN,NaN,SAGE NlaIII
2,GSM3,testis a,RNA,Drosophila melanogaster,1.0,GPL5,GSE462,NaN,NaN,NaN,Brian Oliver,2000-10-18,NaN,NaN,NaN,NaN,spotted DNA/cDNA
3,GSM4,testis b,RNA,Drosophila melanogaster,1.0,GPL5,GSE462,NaN,NaN,NaN,Brian Oliver,2000-10-18,NaN,NaN,NaN,NaN,spotted DNA/cDNA
4,GSM5,male a,RNA,Drosophila melanogaster,1.0,GPL5,GSE462,NaN,NaN,NaN,Brian Oliver,2000-10-18,NaN,NaN,NaN,NaN,spotted DNA/cDNA


In [29]:
# Add a column tagging each accession as GEO or SRA

repoList = []

for i in pmcData['converted_accession']:
    if(type(i) == str):
        if('GSE' in i or 'GPL' in i):
            repoList.append('GEO')
        elif('SRP' in i or 'ERP' in i or 'DRP' in i):
            repoList.append('SRA')
        else:
            repoList.append(np.NaN)
    else:
        repoList.append(np.NaN)

pmcData['repository'] = repoList

In [30]:
pmcData.head()

,journal,pmc_ID,accession,pmc_date,converted_accession,repository
1,Genetics,PMC3606103,GSE44020,2013-04-01,GSE44020,GEO
2,Genetics,PMC3606103,SRX217718,2013-04-01,SRP017961,SRA
3,Genetics,PMC3606103,GSM107672,2013-04-01,GSE4765,GEO
4,Nature,PMC3936678,GSE45494,2013-07-11,GSE45494,GEO
5,Nature,PMC3936678,GSM110564,2013-07-11,GSE4918,GEO


In [ ]:
# add column for paper publish date
#pmc_dates = pd.read_csv('/content/drive/MyDrive/reusability/preFilterMatrix_combined.csv')

#pmcData = pd.merge(pmcData, pmc_dates, how = 'left', on = 'pmc_ID')
#pmcData = pmcData.rename(columns = {'date': 'pmc_date'})

In [31]:
pmcData.head()

,journal,pmc_ID,accession,pmc_date,converted_accession,repository
1,Genetics,PMC3606103,GSE44020,2013-04-01,GSE44020,GEO
2,Genetics,PMC3606103,SRX217718,2013-04-01,SRP017961,SRA
3,Genetics,PMC3606103,GSM107672,2013-04-01,GSE4765,GEO
4,Nature,PMC3936678,GSE45494,2013-07-11,GSE45494,GEO
5,Nature,PMC3936678,GSM110564,2013-07-11,GSE4918,GEO


In [32]:
sraAttributes.head()

,Run,ReleaseDate,bases,avgLength,Experiment,LibraryStrategy,LibrarySelection,LibrarySource,Platform,Model,SRAStudy,BioProject,Sample,ScientificName,Submission
0,SRR7968967,2018-10-08,4538740500,150,SRX4802362,ChIP-Seq,ChIP,GENOMIC,ILLUMINA,Illumina HiSeq 4000,SRP163642,PRJNA494833,SRS3881321,Homo sapiens,SRA789647
1,SRR22439147,2023-12-01,16725226500,300,SRX18408150,RNA-Seq,RANDOM PCR,TRANSCRIPTOMIC,ILLUMINA,Illumina NovaSeq 6000,SRP410343,PRJNA899343,SRS15891322,Homo sapiens,SRA1547816
2,SRR22459658,2023-12-01,7968134424,152,SRX18426201,miRNA-Seq,RT-PCR,TRANSCRIPTOMIC,ILLUMINA,Illumina NovaSeq 6000,SRP410343,PRJNA899343,SRS15891322,Homo sapiens,SRA1549603
3,SRR22439146,2023-12-01,15242483700,300,SRX18408151,RNA-Seq,RANDOM PCR,TRANSCRIPTOMIC,ILLUMINA,Illumina NovaSeq 6000,SRP410343,PRJNA899343,SRS15891323,Homo sapiens,SRA1547816
4,SRR22459657,2023-12-01,5266201272,152,SRX18426202,miRNA-Seq,RT-PCR,TRANSCRIPTOMIC,ILLUMINA,Illumina NovaSeq 6000,SRP410343,PRJNA899343,SRS15891323,Homo sapiens,SRA1549603


In [33]:
# Get every factor we're interested in from our tables of GEO and SRA metadata...

# take a slice of the GEO and SRA attribute tables with only the info we want
slicedGEOAtt = geoAttributes[['Series', 'Release_Date', 'Technology', 'Taxonomy', 'Sample Type', 'Series Type']]
slicedGEOAtt.columns = ['converted_accession', 'geoRelease', 'geoHardware', 'geoSpecies', 'geo_sample_type', 'Series_Type']
slicedGEOAtt = slicedGEOAtt.drop_duplicates(subset = ['converted_accession'])

slicedSRAAtt = sraAttributes[['SRAStudy', 'ReleaseDate', 'Model', 'LibrarySource',
                              'LibraryStrategy', 'ScientificName',
                              'bases', 'avgLength']]
slicedSRAAtt.columns = ['converted_accession', 'sraRelease', 'sraHardware', 'LibrarySource',
                        'sraLibrary_strategy', 'sraSpecies',
                        'sraBases', 'sraAvg_length']
slicedSRAAtt = slicedSRAAtt.drop_duplicates(subset = ['converted_accession'])

In [34]:
slicedGEOAtt2 = slicedGEOAtt

In [35]:
slicedGEOAtt2

,converted_accession,geoRelease,geoHardware,geoSpecies,geo_sample_type,Series_Type
0,GSE506,2000-09-28,SAGE NlaIII,Homo sapiens,SAGE,NaN
2,GSE462,2000-10-18,spotted DNA/cDNA,Drosophila melanogaster,RNA,NaN
10,GSE1,2001-01-24,spotted DNA/cDNA,Homo sapiens,RNA,NaN
48,GSE2,2001-04-26,spotted DNA/cDNA,Mus musculus,RNA,NaN
53,GSE3,2001-07-19,spotted DNA/cDNA,Homo sapiens,RNA,NaN
...,...,...,...,...,...,...
1276653,GSE267405,"May 17, 2024",NaN,Homo sapiens,NaN,Expression profiling by high throughput sequen...
1276654,GSE267438,"May 17, 2024",NaN,Homo sapiens,NaN,Expression profiling by high throughput sequen...
1276655,GSE267595,"May 17, 2024",NaN,Homo sapiens,NaN,Methylation profiling by genome tiling array
1276656,GSE267653,"May 17, 2024",NaN,Gallus gallus,NaN,Expression profiling by high throughput sequen...


In [36]:
# Define the target strings
target_strings = [
    "Non-coding RNA profiling by array",
    "Non-coding RNA profiling by high throughput sequencing",
    "Expression profiling by high throughput sequencing",
    "Genome binding/occupancy profiling by high throughput sequencing",
    "Genome variation profiling by high throughput sequencing"
]

# Function to clean the 'Series_Type' column
def clean_series_type(series_type):
    if pd.isna(series_type):
        return series_type
    for target in target_strings:
        if target in series_type:
            # If the target string is found, return it
            return target
    # If none of the target strings are found, return the original value
    return series_type

# Apply the function to the 'Series_Type' column
slicedGEOAtt['Series_Type'] = slicedGEOAtt['Series_Type'].apply(clean_series_type)


In [37]:
# Replace NaN in 'geoHardware' with corresponding 'Series_Type' values
slicedGEOAtt['geoHardware'] = slicedGEOAtt.apply(
    lambda row: row['Series_Type'] if pd.isna(row['geoHardware']) and pd.notna(row['Series_Type']) else row['geoHardware'],
    axis=1
)
slicedGEOAtt

,converted_accession,geoRelease,geoHardware,geoSpecies,geo_sample_type,Series_Type
0,GSE506,2000-09-28,SAGE NlaIII,Homo sapiens,SAGE,NaN
2,GSE462,2000-10-18,spotted DNA/cDNA,Drosophila melanogaster,RNA,NaN
10,GSE1,2001-01-24,spotted DNA/cDNA,Homo sapiens,RNA,NaN
48,GSE2,2001-04-26,spotted DNA/cDNA,Mus musculus,RNA,NaN
53,GSE3,2001-07-19,spotted DNA/cDNA,Homo sapiens,RNA,NaN
...,...,...,...,...,...,...
1276653,GSE267405,"May 17, 2024",Expression profiling by high throughput sequen...,Homo sapiens,NaN,Expression profiling by high throughput sequen...
1276654,GSE267438,"May 17, 2024",Expression profiling by high throughput sequen...,Homo sapiens,NaN,Expression profiling by high throughput sequen...
1276655,GSE267595,"May 17, 2024",Methylation profiling by genome tiling array,Homo sapiens,NaN,Methylation profiling by genome tiling array
1276656,GSE267653,"May 17, 2024",Expression profiling by high throughput sequen...,Gallus gallus,NaN,Expression profiling by high throughput sequen...


In [38]:
# special case for GEO: make an educated guess on library strategy based on hardware
# These guesses are based on manually checking GEO series IDs that corresponded to various types of hardware

gc = Counter(slicedGEOAtt['geoHardware'])
ls_guesses = pd.DataFrame.from_dict(gc, orient='index').reset_index()
ls_guesses.columns = ['hardware', 'use_count']
ls_guesses = ls_guesses.drop(labels = ['use_count'], axis = 1)

ls = []

for i in ls_guesses['hardware']:
    if(i == 'high-throughput sequencing' or i == 'Non-coding RNA profiling by array'
       or i == 'Non-coding RNA profiling by high throughput sequencing'
       or i == 'Expression profiling by high throughput sequencing' or i == 'Genome binding/occupancy profiling by high throughput sequencing'
       or i == 'Genome variation profiling by high throughput sequencing'):
        ls.append('RNA-Seq')
    elif(i == 'SAGE NlaIII' or i == 'spotted DNA/cDNA' or i == 'SAGE Sau3A' or i == 'SAGE Rsal' or i == 'SAGE Sau3A'
         or i == 'in situ oligonucleotide' or i == 'spotted oligonucleotide'
         or i == 'antibody' or i == 'MPSS' or i == 'oligonucleotide beads'
         or i == 'RT-PCR' or i == 'mixed spotted oligonucleotide/cDNA'
         or i == 'spotted peptide or protein' or i == 'MS' or i == 'Other' or i == 'SARST'
         or i == 'tissue'):
        ls.append('Expression_Array')
    else:
        ls.append('Expression_Array')

ls_guesses.loc[:,'geoLibrary_strategy'] = ls

In [39]:
ls_guesses

,hardware,geoLibrary_strategy
0,SAGE NlaIII,Expression_Array
1,spotted DNA/cDNA,Expression_Array
2,in situ oligonucleotide,Expression_Array
3,spotted oligonucleotide,Expression_Array
4,antibody,Expression_Array
...,...,...
251,Genome variation profiling by array;Genome bin...,Expression_Array
252,Genome variation profiling by SNP array;SNP ge...,Expression_Array
253,Methylation profiling by genome tiling array;G...,Expression_Array
254,Other;Protein profiling by protein array,Expression_Array


In [48]:
# merge SRA attributes onto pmcData table
mergedSRA = pd.merge(pmcData, slicedSRAAtt, how = 'left', on = 'converted_accession')
mergedSRA = mergedSRA.drop_duplicates()

# merge GEO attributes onto table of pmcData + SRA Attributes
allFactors = pd.merge(mergedSRA, slicedGEOAtt, how = 'left', on = 'converted_accession')
allFactors = pd.merge(allFactors, ls_guesses, how = 'left', left_on = 'geoHardware', right_on = 'hardware')

allFactors = allFactors.dropna(subset = ['converted_accession'])

In [49]:
allFactors

,journal,pmc_ID,accession,pmc_date,converted_accession,repository,sraRelease,sraHardware,LibrarySource,sraLibrary_strategy,sraSpecies,sraBases,sraAvg_length,geoRelease,geoHardware,geoSpecies,geo_sample_type,Series_Type,hardware,geoLibrary_strategy
0,Genetics,PMC3606103,GSE44020,2013-04-01,GSE44020,GEO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-02-07,high-throughput sequencing,Panagrellus redivivus,SRA,NaN,high-throughput sequencing,RNA-Seq
1,Genetics,PMC3606103,SRX217718,2013-04-01,SRP017961,SRA,2015-07-22,Illumina Genome Analyzer IIx,TRANSCRIPTOMIC,miRNA-Seq,Panagrellus redivivus,918149996,38,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Genetics,PMC3606103,GSM107672,2013-04-01,GSE4765,GEO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006-11-03,in situ oligonucleotide,Mus musculus,RNA,NaN,in situ oligonucleotide,Expression_Array
3,Nature,PMC3936678,GSE45494,2013-07-11,GSE45494,GEO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-03-27,in situ oligonucleotide,Mus musculus,RNA,NaN,in situ oligonucleotide,Expression_Array
4,Nature,PMC3936678,GSM110564,2013-07-11,GSE4918,GEO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2007-11-21,spotted DNA/cDNA,Sus scrofa,RNA,NaN,spotted DNA/cDNA,Expression_Array
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
406978,BMC_Genomics,PMC1868757,GSE7364,2007-04-30,GSE7364,GEO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2007-03-29,in situ oligonucleotide,Homo sapiens,genomic,NaN,in situ oligonucleotide,Expression_Array
406979,BMC_Cancer,PMC1533844,GSE4127,2006-06-30,GSE4127,GEO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006-02-10,in situ oligonucleotide,Homo sapiens,RNA,NaN,in situ oligonucleotide,Expression_Array
406980,PLoS_Genet,PMC1564424,GSE5405,2006-09-15,GSE5405,GEO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006-09-05,in situ oligonucleotide,Homo sapiens,genomic,NaN,in situ oligonucleotide,Expression_Array
406981,Genome_Biol,PMC1929143,GSE1159,2007-05-08,GSE1159,GEO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2004-04-15,in situ oligonucleotide,Homo sapiens,RNA,NaN,in situ oligonucleotide,Expression_Array


In [50]:
allFactors.rename(columns = {'LibrarySource':'SRA_Library'}, inplace = True)
allFactors

,journal,pmc_ID,accession,pmc_date,converted_accession,repository,sraRelease,sraHardware,SRA_Library,sraLibrary_strategy,sraSpecies,sraBases,sraAvg_length,geoRelease,geoHardware,geoSpecies,geo_sample_type,Series_Type,hardware,geoLibrary_strategy
0,Genetics,PMC3606103,GSE44020,2013-04-01,GSE44020,GEO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-02-07,high-throughput sequencing,Panagrellus redivivus,SRA,NaN,high-throughput sequencing,RNA-Seq
1,Genetics,PMC3606103,SRX217718,2013-04-01,SRP017961,SRA,2015-07-22,Illumina Genome Analyzer IIx,TRANSCRIPTOMIC,miRNA-Seq,Panagrellus redivivus,918149996,38,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Genetics,PMC3606103,GSM107672,2013-04-01,GSE4765,GEO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006-11-03,in situ oligonucleotide,Mus musculus,RNA,NaN,in situ oligonucleotide,Expression_Array
3,Nature,PMC3936678,GSE45494,2013-07-11,GSE45494,GEO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-03-27,in situ oligonucleotide,Mus musculus,RNA,NaN,in situ oligonucleotide,Expression_Array
4,Nature,PMC3936678,GSM110564,2013-07-11,GSE4918,GEO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2007-11-21,spotted DNA/cDNA,Sus scrofa,RNA,NaN,spotted DNA/cDNA,Expression_Array
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
406978,BMC_Genomics,PMC1868757,GSE7364,2007-04-30,GSE7364,GEO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2007-03-29,in situ oligonucleotide,Homo sapiens,genomic,NaN,in situ oligonucleotide,Expression_Array
406979,BMC_Cancer,PMC1533844,GSE4127,2006-06-30,GSE4127,GEO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006-02-10,in situ oligonucleotide,Homo sapiens,RNA,NaN,in situ oligonucleotide,Expression_Array
406980,PLoS_Genet,PMC1564424,GSE5405,2006-09-15,GSE5405,GEO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006-09-05,in situ oligonucleotide,Homo sapiens,genomic,NaN,in situ oligonucleotide,Expression_Array
406981,Genome_Biol,PMC1929143,GSE1159,2007-05-08,GSE1159,GEO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2004-04-15,in situ oligonucleotide,Homo sapiens,RNA,NaN,in situ oligonucleotide,Expression_Array


In [51]:
# clean up columns with factor for both SRA and GEO, rearrange columns
allFactors['species'] = allFactors['sraSpecies'].fillna(allFactors['geoSpecies'])
allFactors = allFactors.drop(labels = ['sraSpecies', 'geoSpecies'], axis = 1)

allFactors['hardware'] = allFactors['sraHardware'].fillna(allFactors['geoHardware'])
allFactors = allFactors.drop(labels = ['sraHardware', 'geoHardware'], axis = 1)

allFactors['library_strategy'] = allFactors['sraLibrary_strategy'].fillna(allFactors['geoLibrary_strategy'])
allFactors = allFactors.drop(labels = ['sraLibrary_strategy', 'geoLibrary_strategy'], axis = 1)

allFactors['repository_date'] = allFactors['sraRelease'].fillna(allFactors['geoRelease'])
allFactors = allFactors.drop(labels = ['sraRelease', 'geoRelease'], axis = 1)

allFactors['library_type'] = allFactors['SRA_Library'].fillna(allFactors['geo_sample_type'])
allFactors = allFactors.drop(labels = ['SRA_Library', 'geo_sample_type'], axis = 1)

cols = ['pmc_ID', 'accession', 'converted_accession', 'repository',
        'pmc_date', 'repository_date', 'species',
        'hardware', 'library_strategy', 'sraAvg_length', 'sraBases', 'library_type', 'Series_Type']

allFactors = allFactors[cols]
allFactors

,pmc_ID,accession,converted_accession,repository,pmc_date,repository_date,species,hardware,library_strategy,sraAvg_length,sraBases,library_type,Series_Type
0,PMC3606103,GSE44020,GSE44020,GEO,2013-04-01,2013-02-07,Panagrellus redivivus,high-throughput sequencing,RNA-Seq,NaN,NaN,SRA,NaN
1,PMC3606103,SRX217718,SRP017961,SRA,2013-04-01,2015-07-22,Panagrellus redivivus,Illumina Genome Analyzer IIx,miRNA-Seq,38,918149996,TRANSCRIPTOMIC,NaN
2,PMC3606103,GSM107672,GSE4765,GEO,2013-04-01,2006-11-03,Mus musculus,in situ oligonucleotide,Expression_Array,NaN,NaN,RNA,NaN
3,PMC3936678,GSE45494,GSE45494,GEO,2013-07-11,2013-03-27,Mus musculus,in situ oligonucleotide,Expression_Array,NaN,NaN,RNA,NaN
4,PMC3936678,GSM110564,GSE4918,GEO,2013-07-11,2007-11-21,Sus scrofa,spotted DNA/cDNA,Expression_Array,NaN,NaN,RNA,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
406978,PMC1868757,GSE7364,GSE7364,GEO,2007-04-30,2007-03-29,Homo sapiens,in situ oligonucleotide,Expression_Array,NaN,NaN,genomic,NaN
406979,PMC1533844,GSE4127,GSE4127,GEO,2006-06-30,2006-02-10,Homo sapiens,in situ oligonucleotide,Expression_Array,NaN,NaN,RNA,NaN
406980,PMC1564424,GSE5405,GSE5405,GEO,2006-09-15,2006-09-05,Homo sapiens,in situ oligonucleotide,Expression_Array,NaN,NaN,genomic,NaN
406981,PMC1929143,GSE1159,GSE1159,GEO,2007-05-08,2004-04-15,Homo sapiens,in situ oligonucleotide,Expression_Array,NaN,NaN,RNA,NaN


In [44]:
# Function to update the 'library_strategy' column based on 'library_type'
#def update_library_strategy(row):
#    if pd.notna(row['library_type']) and 'RNA' in row['library_type']:
#        return 'RNA-Seq'
#    return row['library_strategy']

# Apply the function to update the 'library_strategy' column
#allFactors['library_strategy'] = allFactors.apply(update_library_strategy, axis=1)
#allFactors

,pmc_ID,accession,converted_accession,repository,pmc_date,repository_date,species,hardware,library_strategy,sraAvg_length,sraBases,library_type,Series_Type
0,PMC3606103,GSE44020,GSE44020,GEO,2013-04-01,2013-02-07,Panagrellus redivivus,high-throughput sequencing,RNA-Seq,NaN,NaN,SRA,NaN
1,PMC3606103,SRX217718,SRP017961,SRA,2013-04-01,2015-07-22,Panagrellus redivivus,Illumina Genome Analyzer IIx,miRNA-Seq,38,918149996,TRANSCRIPTOMIC,NaN
2,PMC3606103,GSM107672,GSE4765,GEO,2013-04-01,2006-11-03,Mus musculus,in situ oligonucleotide,RNA-Seq,NaN,NaN,RNA,NaN
3,PMC3936678,GSE45494,GSE45494,GEO,2013-07-11,2013-03-27,Mus musculus,in situ oligonucleotide,RNA-Seq,NaN,NaN,RNA,NaN
4,PMC3936678,GSM110564,GSE4918,GEO,2013-07-11,2007-11-21,Sus scrofa,spotted DNA/cDNA,RNA-Seq,NaN,NaN,RNA,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
406978,PMC1868757,GSE7364,GSE7364,GEO,2007-04-30,2007-03-29,Homo sapiens,in situ oligonucleotide,Expression_Array,NaN,NaN,genomic,NaN
406979,PMC1533844,GSE4127,GSE4127,GEO,2006-06-30,2006-02-10,Homo sapiens,in situ oligonucleotide,RNA-Seq,NaN,NaN,RNA,NaN
406980,PMC1564424,GSE5405,GSE5405,GEO,2006-09-15,2006-09-05,Homo sapiens,in situ oligonucleotide,Expression_Array,NaN,NaN,genomic,NaN
406981,PMC1929143,GSE1159,GSE1159,GEO,2007-05-08,2004-04-15,Homo sapiens,in situ oligonucleotide,RNA-Seq,NaN,NaN,RNA,NaN


In [52]:
# save to .csv
allFactors.to_csv(r'/content/drive/MyDrive/reusability/Metadata_Matrix.csv', index = False)